# Deep Learning for Computer Vision: Final Project

## Computer Science: COMS W 4995 011

### Proposal: Due November 7, 2023
### Presentations: Due December 5 and 7, 2023
### Final Report: Due December 8, 2023

### Project Overview

The final project is one of the most important and, hopefully, exciting components of the course. You will have the opportunity to develop a deep learning system of your own choosing. 
You are free to select whatever framework (Pytorch, Tensorflow, etc.) you like, but you need create a report on your project in a Jupyter notebook. You are also free build on publically available models and code, but your report must clearly give attribution for the work of others and must clearly delineate your contributions. Also, half of the class will present their project during the last 2 days. All of the class will prepare videos of their presentation and submit these when the final report is due. 

### Project Proposal

The project description should include the title of the project, participants, a description of the objectives of the project, and a plan for how the project will be completed. The description of the objectives should include modest predictions of the success of the project. The plan for completion should include a description of the training data and how it will be obtained, a discussion of what deep learning framework will be used and why, and a rough description of the planned network architecture.

You are permitted to work together on a project in groups of two or three, but group size must not exceed three participants.  For group projects there must be a clearly delineated division of labor: you should state in the project description and project report who was responsible for which portion of the project. Each student must hand in a separate report. (Students will not necessarily get the same grade for the same project.)

You should mention whether you are simply re-implementing what others have done before but applying to new data or whether you are attempting to do something new to the best of your knowledge. Creative and original projects will be judged more kindly than those that are rehashing something in the existing literature. And projects that include a component in which data is acquired/curated into training and validation sets will be veiwed more favorably than those that simply download an existing data set such as CIFAR-100.

As this is a computer vision course it is expected that your data will be visual, but exceptions might be made if the student is enthusiastic and persuasive enough. The most straightforward project would be to build a system that classifies images into categories. A more difficult project might be to build a system that detects and localizes a type of object within an image. A still more complicated project might involve joining a ConvNet/Vision Transformer with an LSTM/Transformer for a problem (like image captioning) that requires vision and language. But again, creative and original projects will be judged more kindly.  

It is important to scope your project so that you get some working results. Project reports that say "I tried this and this but nothing seemed to work..." are discouraged. Above all, you should demonstrate end-to-end fluency in the basics of deep learning. 

I cannot wait to see the results. Good luck!

##### participants
Jordyn Kim (jk4671), Sun Kim (syk2145)

##### Project Objectives

The objective of this project is to develop a deep learning model to separate single-channel cinematic audio into three distinct components: voice, background music, and sound effects. This task, often referred to as the cocktail fork separation problem, is highly applicable in the entertainment industry and a prominent focus within the field of audio signal processing.

Based on our preliminary research, we expect voice separation to yield promising results, with clear and high-quality separation. Achieving precise separation of music and effects may, however, face some constraints due to (1) the computational overhead of our chosen architecture (RNN-based models), (2) limited available computational resources, and (3) additional time required for preprocessing audio signals into spectrograms.

##### Project Plan

The project will be completed in three main phases:
1.	Initial Setup and Data Preparation (Phase 1)
	- This phase includes setting up the necessary infrastructure, including establishing a Git repository, importing the Divide and Remaster (DnR) dataset, and converting audio data into spectrograms or mel-spectrograms using short-time Fourier transform (STFT).
2.	Voice vs. Non-Voice Separation (Phase 2)
	-	The focus of this phase is to isolate voice from background music and sound effects. We plan to test two different approaches:
	-	Implementing Band-Split RNN (BSRNN): This model, based on this paper https://arxiv.org/abs/2209.15174, employs the loss function combines frequency-domain and time-domain L1 losses and Adam optimizer for training.
	-	Fine-Tuning Pre-trained Models: We also aim to experiment with fine-tuning models pre-trained on musical source separation, Open-Unmix, and Demucs. As noted in this paper https://arxiv.org/pdf/2308.06981, models trained on music-only data tend to perform better in separating voice from non-voice. We can leverage pre-trained models available through the MUSDB18 music database.
3.	Music vs. Sound Effects Differentiation (Phase 3)
	-	In this final phase, we will build on the voice separation results by extending the band-split RNN model to further differentiate background music from sound effects. Similar network structures will be applied, and additional features or model variations may be introduced based on the results from Phase 2.

##### Training Data

We will use the Divide and Remaster (DnR) dataset https://zenodo.org/records/6949108, approximately 200GB in size, which is specifically structured for the cocktail fork separation problem. This dataset includes labeled annotations for music, sound effects, and speech to support structured training.

##### Deep Learning Framework

We plan to use PyTorch for model implementation due to its compatibility with pre-trained models like Demucs and Open-Unmix, as well as its flexibility for custom network structures.

##### Network Architecture

Our network architecture will primarily focus on the Band-Split RNN (BSRNN) model. The initial step in the pipeline involves splitting spectrogram data into sub-bands by frequency, where each band is then normalized and processed through a fully connected dense layer. The second step introduces two RNNs: one handling the frequency domain and the other the time domain using bidirectional LSTMs. The outputs from all bands are then merged to form the final representation.

In case the BSRNN implementation proves too complex, we will experiment with alternative pre-trained models, such as Open-Unmix and Demucs, to address the separation task.

##### Team Responsibilities

-	Data Collection and Preprocessing	
-	Model Implementation
-	Fine-Tuning and Integration of Pre-Trained Models	
-	Evaluation and Optimization

##### Evaluation

We will evaluate model performance using the Signal-to-Noise-and-Distortion Ratio (SDR) metric, a standard measure for assessing audio source separation quality.

### Project Presentations


To allow students to present their work in two class periods, each student will have only 3 minutes, not a second more. We will be strict about the timing, so you should practice your presentation. The key here is to get across three things: what you did, how you did it, and how well it worked. Students working in groups of two will get 6 minutes and groups of three will get 9 minutes. Note only half of the class will present during the last two days of class, but all of the groups will submit videos of their project. The time limit rules for the videos is the same as for the presentations. And the video can simply be a narration over a slideshow, but again each student in a group needs to present/narrate the work they did.  


### Project Reports

The report should be done as a Jupyter Notebook. The report should be a complete description of the objectives of the work, the methods used to solve the problem, experimental evidence of a working system, the code, and clear delineation of what you have done vs. what you are leveraging that others have done. If you have used the work of others YOU MUST INCLUDE ATTRIBUTION by citing this work inline and as part of a "bibliography" at the end. You should describe what worked, what did not, and why. If you are working in a group you need to submit your own report and this report should be clear about what your individual contribution was. It is ok to include your collaborators work in your report, but you must be clear about your section and write this yourself. This project report constitutes a large fraction of your final grade--take it seriously and include enough material and details for us to give you a good grade. If you are having trouble imagining the structure of the report, refer to published research papers in CV as a possible model.

## Training model

In [7]:
import sys
import os

from torch.utils.data import DataLoader
import torch.nn as nn
import torch

import random
import numpy as np
import soundfile as sf
import librosa

from demucs import pretrained
# from demucs.pretrained import get_model
from demucs.apply import apply_model

# Add the source directory to the Python path
source_path = os.path.abspath("./source")
if source_path not in sys.path:
    sys.path.append(source_path)
import importlib


In [8]:
htdemucs = pretrained.get_model('htdemucs') # load pretrained htdemucs

# modify network to have 3 stems output
model = htdemucs.models[0]
model.sources = ['speech', 'music', 'sfx']

# model.channel = 1
# # Modify the first layer of the encoder to accept mono input
# first_encoder_layer = model.encoder[0]
# if isinstance(first_encoder_layer.conv, torch.nn.Conv2d):
#     original_in_channels = first_encoder_layer.conv.in_channels
#     first_encoder_layer.conv = torch.nn.Conv2d(
#         in_channels=2,  # Change to 1 for mono input
#         out_channels=first_encoder_layer.conv.out_channels,
#         kernel_size=first_encoder_layer.conv.kernel_size,
#         stride=first_encoder_layer.conv.stride,
#         padding=first_encoder_layer.conv.padding,
#         bias=first_encoder_layer.conv.bias is not None,
#     )
#     print(f"Encoder: Changed first layer in_channels from {original_in_channels} to 2")

# # Modify the first layer of the temporal encoder (if applicable)
# first_tencoder_layer = model.tencoder[0]
# if isinstance(first_tencoder_layer.conv, torch.nn.Conv1d):
#     original_in_channels = first_tencoder_layer.conv.in_channels
#     first_tencoder_layer.conv = torch.nn.Conv1d(
#         in_channels=1,  # Change to 1 for mono input
#         out_channels=first_tencoder_layer.conv.out_channels,
#         kernel_size=first_tencoder_layer.conv.kernel_size,
#         stride=first_tencoder_layer.conv.stride,
#         padding=first_tencoder_layer.conv.padding,
#         bias=first_tencoder_layer.conv.bias is not None,
#     )
#     print(f"TEncoder: Changed first layer in_channels from {original_in_channels} to 1")


model.decoder[-1].conv_tr = torch.nn.ConvTranspose2d(
    in_channels=48,  
    out_channels=12,  # 3 stems * input channels (2 for stereo)
    kernel_size=(8, 1),
    stride=(4, 1)
)
model.tdecoder[-1].conv_tr = torch.nn.ConvTranspose1d(
    in_channels=48,  
    out_channels=6,
    kernel_size=8,
    stride=4
)
model

HTDemucs(
  (encoder): ModuleList(
    (0): HEncLayer(
      (conv): Conv2d(4, 48, kernel_size=(8, 1), stride=(4, 1), padding=(2, 0))
      (norm1): Identity()
      (rewrite): Conv2d(48, 96, kernel_size=(1, 1), stride=(1, 1))
      (norm2): Identity()
      (dconv): DConv(
        (layers): ModuleList(
          (0): Sequential(
            (0): Conv1d(48, 6, kernel_size=(3,), stride=(1,), padding=(1,))
            (1): GroupNorm(1, 6, eps=1e-05, affine=True)
            (2): GELU(approximate='none')
            (3): Conv1d(6, 96, kernel_size=(1,), stride=(1,))
            (4): GroupNorm(1, 96, eps=1e-05, affine=True)
            (5): GLU(dim=1)
            (6): LayerScale()
          )
          (1): Sequential(
            (0): Conv1d(48, 6, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,))
            (1): GroupNorm(1, 6, eps=1e-05, affine=True)
            (2): GELU(approximate='none')
            (3): Conv1d(6, 96, kernel_size=(1,), stride=(1,))
            (4): GroupNo

In [ ]:
import train 
importlib.reload(train)

# Create output directory to save model
result_dir = "./output/result/notebook"
os.makedirs(result_dir, exist_ok=True)

# Train with Masked loss
train.train_model(
    model=model,
    results_path=result_dir,
    data_path=['/Users/yt/coding/DL4CV/Final/Cinematic_sound_demixer/DnR/dnr_small/tr'],
    valid_path=['/Users/yt/coding/DL4CV/Final/Cinematic_sound_demixer/DnR/dnr_small/cv'],
    num_epochs=10
)


# trained = train.train_model(
#     model, train_loader, val_loader,
#     epochs=1000, learning_rate=1e-4, output_dir="./models/test",
#     device="cpu", early_stopping=None, loss_type="masked"
# )

Instruments: ['speech', 'music', 'sfx']
Metrics for training: SDR. Metric for scheduler: SDR


100%|██████████| 12/12 [00:02<00:00,  5.73it/s]


Found tracks in dataset: 12
CUDA is not available. Running training on CPU.
Patience: 2 Reduce factor: 0.95 Batch size: 6
Epoch: 1/10


Training Epoch 1: 100%|██████████| 1/1 [01:15<00:00, 75.91s/it, loss=28, avg_loss=28]


Training Loss: 0.280396
Found 2 mixtures for validation.


Average SDR for speech: -3.0198 dB (Std: 0.0022 dB)
Average SDR for music: -3.3848 dB (Std: 0.0338 dB)
Average SDR for sfx: -3.1544 dB (Std: 0.0646 dB)
Average SDR over all tracks: -3.1864 dB
Elapsed time: 204.94 sec
Validation SDR: -3.1864
New best model saved: ./output/result/notebook/best_model_epoch_1_-3.1864.ckpt
Epoch: 2/10


Training Epoch 2:   0%|          | 0/1 [00:00<?, ?it/s]

## ZZ

In [22]:
import dataset
importlib.reload(dataset)

def get_subset(dataset, subset_fraction=0.5):
    total_samples = len(dataset)
    subset_size = int(total_samples * subset_fraction)
    indices = random.sample(range(total_samples), subset_size)
    return torch.utils.data.Subset(dataset, indices)

# Initialize dataset
dataset_path = "DnR/dnr_small"  # Use small dataset for local testing. Replace with the full dataset 
train_path = os.path.join(dataset_path, "tr")  # Folder containing training data
val_path = os.path.join(dataset_path, "cv")  # Folder containing validation data
test_path = os.path.join(dataset_path, "tt")  # Folder containing test data

# Initialize datasets
chunk_size = 132300
train_dataset = dataset.DnR_Dataset(root_dir=train_path, sample_rate=44100, chunk_size=chunk_size)
val_dataset = dataset.DnR_Dataset(root_dir=val_path, sample_rate=44100, chunk_size=chunk_size)
test_dataset = dataset.DnR_Dataset(root_dir=test_path, sample_rate=44100, chunk_size=chunk_size)

# Use only a fraction of the data for faster training
# train_dataset = get_subset(train_dataset, 0.001) 
# val_dataset = get_subset(val_dataset, 0.1)  
# test_dataset = get_subset(test_dataset)  

# Initialize DataLoaders
batch_size = 2 # update batch size when using GPU
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [26]:
import train 
importlib.reload(train)

# Create output directory to save model
output_dir = "./models/half_dataset"
os.makedirs(output_dir, exist_ok=True)

# Train with Masked loss
trained = train.train_model(
    model, train_loader, val_loader,
    epochs=1000, learning_rate=1e-4, output_dir="./models/test",
    device="cpu", early_stopping=None, loss_type="masked"
)

--------------------
Epoch 1/1000


Training Loss: 0.09759910
Validation Loss: 0.12009210
Average SDR for Stem 0: -3.72681355 dB
Average SDR for Stem 1: -0.10373989 dB
Average SDR for Stem 2: -11.20057297 dB
Overall Average SDR: -5.01037550 dB
--------------------
Epoch 2/1000


Training Loss: 0.13706639
Validation Loss: 0.04634798
Average SDR for Stem 0: -21.30552292 dB
Average SDR for Stem 1: -2.34668994 dB
Average SDR for Stem 2: -0.91916823 dB
Overall Average SDR: -8.19046021 dB
--------------------
Epoch 3/1000


Training Loss: 0.10451783
Validation Loss: 0.07554083
Average SDR for Stem 0: -4.30159807 dB
Average SDR for Stem 1: -4.25910091 dB
Average SDR for Stem 2: -8.33948231 dB
Overall Average SDR: -5.63339376 dB
--------------------
Epoch 4/1000


Training Loss: 0.12946711
Validation Loss: 0.16043242
Average SDR for Stem 0: -0.02839401 dB
Average SDR for Stem 1: -0.09067234 dB
Average SDR for Stem 2: -4.07820988 dB
Overall Average SDR: -1.39909208 dB
--------------------
Epoch 5/1000


...

Training Loss: 0.06811619
Validation Loss: 0.04848431
Average SDR for Stem 0: -1.58816051 dB
Average SDR for Stem 1: -0.92924786 dB
Average SDR for Stem 2: -0.27088183 dB
Overall Average SDR: -0.92943007 dB
--------------------
Epoch 549/1000


Training Loss: 0.09555925
Validation Loss: 0.05883867
Average SDR for Stem 0: -0.77599078 dB
Average SDR for Stem 1: -0.07478768 dB
Average SDR for Stem 2: -7.14799309 dB
Overall Average SDR: -2.66625714 dB
--------------------
Epoch 550/1000


Training Loss: 0.06048410
Validation Loss: 0.19168798
Average SDR for Stem 0: -5.02903795 dB
Average SDR for Stem 1: -4.29851532 dB
Average SDR for Stem 2: -1.77011395 dB
Overall Average SDR: -3.69922233 dB
--------------------
Epoch 551/1000


Training Loss: 0.13203456
Validation Loss: 0.05823157
Average SDR for Stem 0: -5.99278784 dB
Average SDR for Stem 1: -0.21091433 dB
Average SDR for Stem 2: -4.99490690 dB
Overall Average SDR: -3.73286939 dB
--------------------
Epoch 552/1000


KeyboardInterrupt: 

In [ ]:
import train 
importlib.reload(train)

# Create output directory to save model
output_dir = "./models/half_dataset"
os.makedirs(output_dir, exist_ok=True)

# Train with Masked loss
trained = train.train_model(
    model, train_loader, val_loader,
    epochs=50, learning_rate=1e-4, output_dir="./models/",
    device="cpu", early_stopping=train.EarlyStopping(), loss_type="si_snr"
)

--------------------
Epoch 1/50


100%|██████████| 2/2 [01:28<00:00, 44.05s/it]


Training Loss: 52.52687454
Validation Loss: 49.61462402
Average SDR: -4.55867434 dB
Validation loss improved to 49.61462402, model saved.
--------------------
Epoch 2/50


100%|██████████| 2/2 [01:27<00:00, 43.51s/it]


Training Loss: 50.15989685
Validation Loss: 53.16985321
Average SDR: -9.15587711 dB
No improvement in validation loss. Counter: 1/10
--------------------
Epoch 3/50


100%|██████████| 2/2 [01:26<00:00, 43.42s/it]


Training Loss: 53.36610794
Validation Loss: 44.91479874
Average SDR: -6.11486769 dB
Validation loss improved to 44.91479874, model saved.
--------------------
Epoch 4/50


100%|██████████| 2/2 [01:48<00:00, 54.12s/it]


Training Loss: 51.09166527
Validation Loss: 51.18777084
Average SDR: -9.71933079 dB
No improvement in validation loss. Counter: 1/10
--------------------
Epoch 5/50


 50%|█████     | 1/2 [00:49<00:49, 49.95s/it]

In [ ]:
import train 
importlib.reload(train)

# Create output directory to save model
output_dir = "./models/half_dataset"
os.makedirs(output_dir, exist_ok=True)

# Train with Masked loss
trained = train.train_model(
    model, train_loader, val_loader,
    epochs=50, learning_rate=1e-4, output_dir="./models/",
    device="cpu", early_stopping=train.EarlyStopping(), loss_type="si_snr"
)

--------------------
Epoch 1/50


100%|██████████| 2/2 [01:16<00:00, 38.44s/it]


Training Loss: 51.11145973
Validation Loss: 49.64332581
Average SDR: -0.40868154 dB
Validation loss improved to 0.40868154, model saved.
--------------------
Epoch 2/50


100%|██████████| 2/2 [01:19<00:00, 39.58s/it]


Training Loss: 50.30214500
Validation Loss: 51.68160629
Average SDR: -12.83787632 dB
No improvement in validation loss. Counter: 1/10
--------------------
Epoch 3/50


...

Training Loss: 51.04137611
Validation Loss: 55.67629242
Average SDR: -13.43802261 dB
No improvement in validation loss. Counter: 8/10
--------------------
Epoch 10/50


100%|██████████| 2/2 [01:31<00:00, 45.84s/it]


Training Loss: 47.23505402
Validation Loss: 52.54701996
Average SDR: -7.13948727 dB
No improvement in validation loss. Counter: 9/10
--------------------
Epoch 11/50


 50%|█████     | 1/2 [00:57<00:57, 57.98s/it]


KeyboardInterrupt: 

In [ ]:
import train 
importlib.reload(train)

# Create output directory to save model
output_dir = "./models/half_dataset"
os.makedirs(output_dir, exist_ok=True)

# Train with Masked loss
trained = train.train_model(
    model, train_loader, val_loader,
    epochs=50, learning_rate=1e-4, output_dir="./models/",
    device="cpu", early_stopping=train.EarlyStopping(), loss_type="masked"
)

--------------------
Epoch 1/50


100%|██████████| 2/2 [01:14<00:00, 37.07s/it]


Training Loss: 0.00044986
Validation Loss: 0.00037341
Average SDR: -8.92233753 dB
Validation loss improved to 0.00037341, model saved.
--------------------
Epoch 2/50


100%|██████████| 2/2 [01:13<00:00, 36.63s/it]


Training Loss: 0.00046402
Validation Loss: 0.00053507
Average SDR: -5.22554255 dB
No improvement in validation loss. Counter: 1/10
--------------------
Epoch 3/50


 50%|█████     | 1/2 [01:14<01:14, 74.30s/it]


KeyboardInterrupt: 

In [ ]:
import train 
importlib.reload(train)

# Create output directory to save model
output_dir = "./models/half_dataset"
os.makedirs(output_dir, exist_ok=True)

# DEVICE = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
DEVICE = "cpu"

model.to(DEVICE)
# Train the model
trained_model = train.train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    early_stopping = train.EarlyStopping(),
    epochs=100,  
    learning_rate=1e-4,  # Learning rate
    output_dir=output_dir,
    device = DEVICE
)

--------------------
Epoch 1/100


100%|██████████| 2/2 [01:17<00:00, 38.76s/it]


Training Loss: 49.27630234
Validation Loss: 47.09437561
Average SDR: -5.00354195 dB
Validation loss improved to 47.09437561, model saved.
--------------------
Epoch 2/100


100%|██████████| 2/2 [01:18<00:00, 39.47s/it]


Training Loss: 48.89335251
Validation Loss: 54.91148758
Average SDR: -8.42761993 dB
No improvement in validation loss. Counter: 1/10
--------------------
Epoch 3/100


...

Training Loss: 50.72283173
Validation Loss: 49.53795624
Average SDR: -0.63632107 dB
No improvement in validation loss. Counter: 3/10
--------------------
Epoch 7/100


100%|██████████| 2/2 [01:27<00:00, 43.66s/it]


Training Loss: 49.56028175
Validation Loss: 56.56599808
Average SDR: -12.88835716 dB
No improvement in validation loss. Counter: 4/10
--------------------
Epoch 8/100


 50%|█████     | 1/2 [00:52<00:52, 52.63s/it]


KeyboardInterrupt: 

In [ ]:
import torch
from demucs import pretrained
from demucs.htdemucs import HTDemucs


# Define the model architecture
htdemucs = pretrained.get_model('htdemucs') # load pretrained htdemucs

# modify network to have 3 stems output
checkpoint_model = htdemucs.models[0]
checkpoint_model.sources = ['speech', 'music', 'sfx']
checkpoint_model.decoder[-1].conv_tr = torch.nn.ConvTranspose2d(
    in_channels=48,
    out_channels=12,  # 3 stems * input channels (2 for stereo)
    kernel_size=(8, 1),
    stride=(4, 1)
)
checkpoint_model.tdecoder[-1].conv_tr = torch.nn.ConvTranspose1d(
    in_channels=48,
    out_channels=6,
    kernel_size=8,
    stride=4
)

# Path to the .ckpt file
ckpt_path = "/Users/yt/coding/DL4CV/Final/Music-Source-Separation-Training/htdemucs_finetuned_500.ckpt"

# Load the checkpoint
checkpoint = torch.load(ckpt_path, map_location=torch.device('cpu'))  # Use 'cuda' if GPU is available

# Check if the checkpoint contains just the state_dict or the full model
if 'state_dict' in checkpoint:
    # Load state_dict into the model
    checkpoint_model.load_state_dict(checkpoint['state_dict'])
    print("Loaded state_dict into the model.")
else:
    # If the checkpoint is the full model
    checkpoint_model.load_state_dict(checkpoint)
    print("Loaded full model directly.")

# Set the model to evaluation mode (if needed)
checkpoint_model.eval()

# (Optional) If optimizer state is in the checkpoint and you need it
if 'optimizer_state_dict' in checkpoint:
    optimizer = torch.optim.Adam(model.parameters())  # Use the correct optimizer
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    print("Loaded optimizer state.")

# Print success
print("Model successfully loaded!")

/var/folders/ry/tyg2vvqs2t30yq4510jl_ns80000gn/T/ipykernel_34058/2916665466.py:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(ckpt_path, map_locat

RuntimeError: Error(s) in loading state_dict for HTDemucs:
	Missing key(s) in state_dict: "channel_upsampler.weight", "channel_upsampler.bias", "channel_downsampler.weight", "channel_downsampler.bias", "channel_upsampler_t.weight", "channel_upsampler_t.bias", "channel_downsampler_t.weight", "channel_downsampler_t.bias". 
	size mismatch for crosstransformer.norm_in.weight: copying a param with shape torch.Size([384]) from checkpoint, the shape in current model is torch.Size([512]).
	size mismatch for crosstransformer.norm_in.bias: copying a param with shape torch.Size([384]) from checkpoint, the shape in current model is torch.Size([512]).
	size mismatch for crosstransformer.norm_in_t.weight: copying a param with shape torch.Size([384]) from checkpoint, the shape in current model is torch.Size([512]).
	size mismatch for crosstransformer.norm_in_t.bias: copying a param with shape torch.Size([384]) from checkpoint, the shape in current model is torch.Size([512]).
	size mismatch for crosst

In [34]:
checkpoint_model = HTDemucs( sources = ['speech', 'music', 'sfx'] )
checkpoint_model.to("cpu")
checkpoint = torch.load(f"/Users/yt/coding/DL4CV/Final/Music-Source-Separation-Training/htdemucs_finetuned_500.ckpt")
checkpoint_model.load_state_dict(checkpoint, map_location=torch.device('cpu'))

/var/folders/ry/tyg2vvqs2t30yq4510jl_ns80000gn/T/ipykernel_34058/1640722872.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(f"/Users/yt/coding/DL

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [27]:
# validate model output
def load_audio(audio_path, sample_rate=44100, device='cpu', channel=1):
    # Load audio with librosa
    audio, sr = librosa.load(audio_path, mono=False, sr=sample_rate)

    # Handle mono audio by duplicating the channel
    if len(audio.shape) != channel:  # Mono audio
        audio = np.stack([audio, audio], axis=0)  # Convert to stereo

    audio = np.expand_dims(audio, axis=0)  # Shape: [1, samples, channels]
    audio = torch.from_numpy(audio).float().to(device)

    return audio
audio_tensor = load_audio("DnR/dnr_small/tr/106/mix.wav") # change to correct path
out = apply_model(trained_model, audio_tensor, shifts=1, overlap=0.8)[0].cpu().numpy()
print(out.shape)

for i, source in enumerate(out[:,:,:]):
    source = np.array(source).mean(axis=0)  # Convert tensor to numpy array
    sf.write(f'small_train_masked_loss{i}.wav', source.T, 44100)  # Transpose if needed for correct shape

NameError: name 'trained_model' is not defined

In [ ]:
%skip
# reference: https://github.com/ZFTurbo/Music-Source-Separation-Training/blob/730d162b2ef31a1bba5d4a7d40ae914459a011b8/utils.py#L221
# TODO: move evaluation to separate module
def sdr(references, estimates):
    # compute SDR for one song
    delta = 1e-7  # avoid numerical errors
    num = np.sum(np.square(references), axis=(1, 2))
    den = np.sum(np.square(references - estimates), axis=(1, 2))
    num += delta
    den += delta
    return 10 * np.log10(num / den)

def si_sdr(reference, estimate):
    eps = 1e-07
    scale = np.sum(estimate * reference + eps, axis=(0, 1)) / np.sum(reference**2 + eps, axis=(0, 1))
    scale = np.expand_dims(scale, axis=(0, 1))  # shape - [50, 1]
    reference = reference * scale
    sisdr = np.mean(10 * np.log10(np.sum(reference**2, axis=(0, 1)) / (np.sum((reference - estimate)**2, axis=(0, 1)) + eps) + eps))
    return sisdr

references_path_prefix = "DnR/dnr_small/tr/106"
predicted_file_prefix = "small_train_masked_loss"
references = []
predicted = []
for i, source in enumerate(model.sources):
    references += load_audio(f"{references_path_prefix}/{source}.wav")
    predicted += load_audio(f"{predicted_file_prefix}{i}.wav")

references = np.stack(references)  # Shape: [num_sources, channels, samples]
predicted = np.stack(predicted)  # Shape: [num_sources, channels, samples]

# Compute SDR
sdr_score = sdr(references, predicted)
mean_sdr = np.mean(sdr_score)
print(f"Mean SDR: {mean_sdr}")

si_mean_sdr = np.mean(si_sdr(references, predicted))
print(f"Mean si_SDR: {si_mean_sdr}")

NameError: name 'load_audio' is not defined